In [0]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as npr
import math
from numpy.linalg import inv
import scipy
import scipy.linalg   # SciPy Linear Algebra Library

## Etapa 1

In [3]:
#Etapa 1
A = np.matrix([[3,2,4],[1,1,2],[4,3,2]],dtype='float')

m21 = A[1,0]/A[0,0]
m31 = A[2,0]/A[0,0]

m0 = np.matrix([[1,0,0],[-m21,1,0],[-m31,0,1]],dtype='float')

A1 = m0*A

print(A1)

[[ 3.          2.          4.        ]
 [ 0.          0.33333333  0.66666667]
 [ 0.          0.33333333 -3.33333333]]


## Etapa 2

In [4]:
#Etapa 2

m32 = A1[2,1]/A1[1,1]

m1 = np.matrix([[1,0,0],[0,1,0],[0,-m32,1]],dtype='float')

A2 = m1*A1

print(A2)

[[ 3.00000000e+00  2.00000000e+00  4.00000000e+00]
 [ 0.00000000e+00  3.33333333e-01  6.66666667e-01]
 [ 0.00000000e+00  5.55111512e-17 -4.00000000e+00]]


## Etapa 3 - Verificar b2

In [5]:
#Etapa 3

b = np.array([1,2,3],dtype='float')

b1 = np.matmul(m0.data,b)

b2 = np.matmul(m1.data,b1)

print(b2)

[ 1.00000000e+00  1.66666667e+00 -4.44089210e-16]


## Atividade 5

In [194]:
def solveUpperTriangular(A,b):
    n = A.shape[0]
    x = np.zeros(n)
    
    #x[n-1] = b[n-1]/A[n-1,n-1]
    
    for k in range(n-1,-1,-1):
        s = 0
        for j in range(k,n):
            s += A[k,j]*x[j]
        x[k] = (b[k] - s)/A[k,k]
            
    return x

def solveLowerTriangular(A, b):
    n = A.shape[0]
    x = np.zeros(n)
    for i in range(n):
        x[i] = (b[i] - (x*A[i]).sum())/A[i][i]
    return x

def decompose_LU(A,b):  
    # Cria uma matriz U, que é uma cópia de A
    U = np.copy(A)
    n = np.shape(U)[0]  
    
    
    # L é uma matriz identidade de tamanho n x n
    L = np.eye(n)
    
    # Eliminação de Gauss sobre a matriz U ajustando num sistema triangular superior.
    for j in range(n-1):  
        for i in range(j+1,n):
            L[i,j] = U[i,j]/U[j,j]  
            U[i,j] = 0
            for k in range(j+1,n):  
                U[i,k] = U[i,k] - L[i,j]*U[j,k]  
            
    # Cálculo L.b = y
    y = solveLowerTriangular(L,b)
    #y = np.linalg.solve(L,b)
    
    # Obtido y, faça U.y = x, onde x é a solução final do sistema.
    x = solveUpperTriangular(U,y)
    
    print('Matrix L:\n',L)
    print('\nMatrix U:\n',U)
    print('\nSolução y:\n',y)
    print('\nSolução x:\n',x)
    

A = np.array([[3,2,4],[1,1,2],[4,3,2]], dtype='float')
b = np.array([1,2,3], dtype='float')

#A = np.matrix([[5,4],[3,-6]], dtype='float')
#b = np.array([1,2], dtype='float')

#A = np.matrix([[4,3],[6,3]], dtype='float')
#b = np.array([0,0])

decompose_LU(A,b)

Matrix L:
 [[1.         0.         0.        ]
 [0.33333333 1.         0.        ]
 [1.33333333 1.         1.        ]]

Matrix U:
 [[ 3.          2.          4.        ]
 [ 0.          0.33333333  0.66666667]
 [ 0.          0.         -4.        ]]

Solução y:
 [ 1.00000000e+00  1.66666667e+00 -4.44089210e-16]

Solução x:
 [-3.00000000e+00  5.00000000e+00  1.11022302e-16]


## Fatoração LU com pivoteamento parcial

In [7]:
P = np.matrix([[0,1,0],[0,0,1],[1,0,0]], dtype='float')
A = np.matrix([[3,1,4],[1,5,9],[2,6,5]], dtype='float')

print(P @ A)

[[1. 5. 9.]
 [2. 6. 5.]
 [3. 1. 4.]]


In [291]:
def LU_partial_decomposition(A,b):
    n = np.shape(A)[0]
    m = np.shape(A)[1]
    
    U = np.copy(A)
    P = np.eye(n)
    L = np.eye(n)
    
    # P_final e L_final são as matrizes P e F que serão modificadas ao longo do processo, ou seja, as matrizes finais da operação
    P_final = np.eye(n)
    L_final = np.zeros((n,n))
    
    for k in range(n-1):
        #Verifica qual o maior elemento da coluna
        r = np.argmax(abs(U[k:,k]))
        r += k 
        
        if (r != k):
            P = np.eye(n)
            P[[r,k],k:n] = P[[k,r],k:n]
            U[[r,k],k:n] = U[[k,r],k:n] 
            
            print("\nEstagio de U:\n",U)
            P_final = np.dot(P,P_final)
            
            #A' = P * A
            print("\nEstagio de P:\n",P_final)
            L_final = np.dot(P,L_final)
            
        L = np.eye(n)
        
        #Seleciona o pivô, então executa a permutação que zera os elementos abaixo do pivô
        for j in range(k+1,n):
            L[j,k] = -(U[j,k] / U[k,k])
            L_final[j,k] = (U[j,k] / U[k,k])
            print("\nEstagio de L:\n",L_final)
            
        U = np.dot(L,U)
        print("\nEstagio de U:\n",U)
    
    #Preenche a diagonal principal com elementos 1
    np.fill_diagonal(L_final, 1)
    
    # i) Ly = Pb, onde:
    Pb = P_final @ b
    y = solveLowerTriangular(L_final,Pb)
    
    # ii) Ux = y
    x = solveUpperTriangular(U,y)
    
    
    print("\nResultados finais\n")
    print("Matrix Identidade final (P):\n",P_final)
    print("\nFator L:\n",L_final)
    print("\nFator U:\n",U)
    print("\nSolução y:\n",y)
    print("\nSolução x:\n",x)
    

A = np.matrix([[3,-4,1],[1,2,2],[4,0,-3]], dtype='float')
b = np.array([9,3,-2], dtype='float')

LU_partial_decomposition(A,b)


Estagio de U:
 [[ 4.  0. -3.]
 [ 1.  2.  2.]
 [ 3. -4.  1.]]

Estagio de P:
 [[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]

Estagio de L:
 [[0.   0.   0.  ]
 [0.25 0.   0.  ]
 [0.   0.   0.  ]]

Estagio de L:
 [[0.   0.   0.  ]
 [0.25 0.   0.  ]
 [0.75 0.   0.  ]]

Estagio de U:
 [[ 4.    0.   -3.  ]
 [ 0.    2.    2.75]
 [ 0.   -4.    3.25]]

Estagio de U:
 [[ 4.    0.   -3.  ]
 [ 0.   -4.    3.25]
 [ 0.    2.    2.75]]

Estagio de P:
 [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]

Estagio de L:
 [[ 0.    0.    0.  ]
 [ 0.75  0.    0.  ]
 [ 0.25 -0.5   0.  ]]

Estagio de U:
 [[ 4.     0.    -3.   ]
 [ 0.    -4.     3.25 ]
 [ 0.     0.     4.375]]

Resultados finais

Matrix Identidade final (P):
 [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]

Fator L:
 [[ 1.    0.    0.  ]
 [ 0.75  1.    0.  ]
 [ 0.25 -0.5   1.  ]]

Fator U:
 [[ 4.     0.    -3.   ]
 [ 0.    -4.     3.25 ]
 [ 0.     0.     4.375]]

Solução y:
 [-2.   10.5   8.75]

Solução x:
 [ 1. -1.  2.]
